In [1]:
from Bio import SeqIO
import pandas as pd
from collections import Counter

In [2]:
def counter_to_dataframe(counter):
    df = pd.DataFrame.from_dict(counter, orient='index', columns=['Count'])
    df.index.name = 'Value'
    df.reset_index(inplace=True)
    return df

In [7]:
fasta_file = "fasta.aln" ######
contigs = {}

with open(fasta_file, "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        contigs[record.id] = record

In [8]:
new_order=pd.read_csv('reorder.csv')['reorder'].dropna().to_list()

reordered_contigs = {}
for contig_id in new_order:
    if contig_id in contigs:
        reordered_contigs[contig_id] = contigs[contig_id]

output_file = "re_ordered.aln"   #######

with open(output_file, "w") as handle:
    for contig_id, contig_record in reordered_contigs.items():
        SeqIO.write(contig_record, handle, "fasta")

print("Contigs reordered and saved to", output_file)


Contigs reordered and saved to re_ordered.aln


Process SNP Distance

In [6]:
SNP_file=pd.read_excel('SNP_distance.xlsx',index_col=0,sheet_name='Asia_isolate')
SNP_file

indices, columns = [], []
value=[]

for index, row in SNP_file.iterrows():
    for column in SNP_file.columns:
        if row[column] <= 100:
            indices.append(index)
            columns.append(column)
            value.append(row[column])

result_df = pd.DataFrame({'Index': indices, 'Column': columns,'value':value})
result_df = result_df[result_df['value']!=0]
result_df
result_df.to_csv('Asia_SNP_result.csv',index=False)

手工处理，打上省份标签

In [4]:
file=pd.read_excel('Global.xlsx',index_col=0)
file

,time1,location1,Column,time2,location2,value
Index,,,,,,
E388-1.clean.fq,2023,Aisa,SAL_PA7693AA,2012,Asia,97
E388-1.clean.fq,2023,Aisa,SAL_PA7757AA,2008,Asia,93
E388-1.clean.fq,2023,Aisa,SAL_PA7983AA,2007,Asia,94
E388-1.clean.fq,2023,Aisa,TH184,2022,Asia,50
E388-2.clean.fq,2023,Aisa,SAL_PA7693AA,2012,Asia,97
...,...,...,...,...,...,...
M90-1.clean.fq,2022,Aisa,X90.clean.fq,2022,Asia,65
M90-1.clean.fq,2022,Aisa,X93.clean.fq,2022,Asia,67
M90-1.clean.fq,2022,Aisa,X97.clean.fq,2022,Asia,65


In [9]:
year=file['Spread'].dropna().to_list()
year=Counter(year)
year=counter_to_dataframe(year)
year.to_csv('spread_count.csv',index=False, encoding='gbk')

predict

In [2]:
file=pd.read_excel('Asia_SNP_result.xlsx',index_col=0)
file

,time1,province1,Column,time2,province2,value
Index,,,,,,
E388-1.clean.fq,2023,Zhejiang,SAL_PA7693AA,2012,Anhui,97
E388-1.clean.fq,2023,Zhejiang,SAL_PA7757AA,2008,Henan,93
E388-1.clean.fq,2023,Zhejiang,SAL_PA7983AA,2007,Jiangsu,94
E388-1.clean.fq,2023,Zhejiang,TH184,2022,Jiangxi,50
E388-2.clean.fq,2023,Zhejiang,SAL_PA7693AA,2012,Anhui,97
...,...,...,...,...,...,...
M90-1.clean.fq,2022,Zhejiang,X90.clean.fq,2022,Zhejiang,65
M90-1.clean.fq,2022,Zhejiang,X93.clean.fq,2022,Zhejiang,67
M90-1.clean.fq,2022,Zhejiang,X97.clean.fq,2022,Zhejiang,65


In [3]:
file_time1=file['time1'].dropna().to_list()
file_time2=file['time2'].dropna().to_list()
file_province1=file['province1'].dropna().to_list()
file_province2=file['province2'].dropna().to_list()

Spread=[]
Time=[]

timer=0

while timer < 2526:
    time1=file_time1[timer]
    time2=file_time2[timer]
    if time1<time2:
        spread=file_province1[timer]+'-'+file_province2[timer]
        time=str(file_time1[timer])+'-'+str(file_time2[timer])
        Spread.append(spread)
        Time.append(time)
    else:
        spread=file_province2[timer]+'-'+file_province1[timer]
        time=str(file_time2[timer])+'-'+str(file_time1[timer])
        Spread.append(spread)
        Time.append(time)
    timer+=1
    
print(len(Spread))
print(len(Time))

2526
2526


In [6]:
file['Time']=Time
file['Spread']=Spread
file.to_csv('2_predict_final_global.csv')

In [6]:
file=pd.read_csv('2_predict_final_Asia.csv')
file

,Index,time1,province1,Column,time2,province2,value,Time,Spread
0,E388-1.clean.fq,2023,Zhejiang,SAL_PA7693AA,2012,Anhui,97,2012-2023,Anhui-Zhejiang
1,E388-1.clean.fq,2023,Zhejiang,SAL_PA7757AA,2008,Henan,93,2008-2023,Henan-Zhejiang
2,E388-1.clean.fq,2023,Zhejiang,SAL_PA7983AA,2007,Jiangsu,94,2007-2023,Jiangsu-Zhejiang
3,E388-1.clean.fq,2023,Zhejiang,TH184,2022,Jiangxi,50,2022-2023,Jiangxi-Zhejiang
4,E388-2.clean.fq,2023,Zhejiang,SAL_PA7693AA,2012,Anhui,97,2012-2023,Anhui-Zhejiang
...,...,...,...,...,...,...,...,...,...
2521,M90-1.clean.fq,2022,Zhejiang,X90.clean.fq,2022,Zhejiang,65,2022-2022,Zhejiang-Zhejiang
2522,M90-1.clean.fq,2022,Zhejiang,X93.clean.fq,2022,Zhejiang,67,2022-2022,Zhejiang-Zhejiang
2523,M90-1.clean.fq,2022,Zhejiang,X97.clean.fq,2022,Zhejiang,65,2022-2022,Zhejiang-Zhejiang
2524,M90-1.clean.fq,2022,Zhejiang,X98.clean.fq,2022,Zhejiang,66,2022-2022,Zhejiang-Zhejiang


In [7]:
year=file['Spread'].dropna().to_list()
year=Counter(year)
year=counter_to_dataframe(year)
year.to_csv('spread_count.csv',index=False, encoding='gbk')